Import the numpy and pandas libraries

In [ ]:
import numpy as np 
import pandas as pd

Create the dataframes I made in the previous steps

In [ ]:
path_coord = 'http://cocl.us/Geospatial_data'
path_neigh ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

l_df_neigh = pd.read_html(path_neigh, header=0,keep_default_na=False) 
df_neigh = l_df_neigh[0]

df_neigh.columns = ['PostalCode','Borough','Neighborhood']
df_neigh.head()

df_neigh1 = df_neigh.query('Borough != "Not assigned"').reset_index(drop=True)

df_neigh2 = df_neigh1.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

df_neigh2.loc[df_neigh2['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = df_neigh2['Borough']

df_coord = pd.read_csv(path_coord)

result = pd.merge(df_neigh2,
                 df_coord,
                 left_on = 'PostalCode',
                 right_on = 'Postal Code',
                 how = 'left')

merge_table = result[['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']]
merge_table.head()

Import the libraries for Geocoding purposes

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  20.80 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00  28.32 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  44.64 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.12 MB/s
vincent-0.4.4- 100% |###################

Use the query to select the areas with the name of Toronto

In [4]:
df_neigh4 = merge_table.query('Borough == ["West Toronto", "East Toronto", "Downtown Toronto"]').reset_index(drop=True)

In [ ]:
Print the first five rows of the dataset

In [15]:
df_neigh4.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
5,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
6,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
7,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
9,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Use geopy library for the latitude and longitude of Toronto, ON.

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


Verifying the dataset in terms of number of Boroughs and Neighborhoods

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_neigh4['Borough'].unique()),
        df_neigh4.shape[0]
    )
)

The dataframe has 3 boroughs and 29 neighborhoods.


Visualize the neighborhoods of Toronto on the map

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh4['Latitude'], df_neigh4['Longitude'], df_neigh4['Borough'], df_neigh4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create dataframe for Downtown Toronto

In [23]:
downtown_data = df_neigh4[df_neigh4['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Obtain geographical coordinates of Downtown Toronto

In [22]:
l_addr = 'Downtown Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Downtown Toronto are 43.653963, -79.387207.


In [ ]:
Maping neighborhoods of downtown Toronto

In [24]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Define Foursquare Credentials

In [25]:
CLIENT_ID = 'DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM' # your Foursquare ID
CLIENT_SECRET = 'JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM
CLIENT_SECRET:JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD


My third neighborhood in Downtown Toronto

In [45]:
downtown_data.loc[3, 'Neighborhood']

'Harbourfront, Regent Park'

Ontain lat & long values for neighborhood

In [46]:
neighborhood_lat = downtown_data.loc[0, 'Latitude'] 
neighborhood_long = downtown_data.loc[0, 'Longitude'] 
neighborhood_name = downtown_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


get the top 50 venues that are in Rosedale within a radius of 1000 meters.

In [48]:
# type your answer here

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM&client_secret=JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD&v=20180605&ll=43.6795626,-79.37752940000001&radius=1000&limit=50'

Viewing results

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c14bbb5f594df27dd10ede3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4adcb343f964a520e32e21e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4adcb343f964a520e32e21e3',
       'location': {'address': '446 Summerhill Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'distance': 764,
        'formattedAddress': ['446 Summerhill Ave (btwn. MacLennan Ave. and Glen Rd.)',
         'Toronto

use function that extracts the category of the venue

In [51]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean up the json and structure it into a pandas dataframe

In [52]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Black Camel,BBQ Joint,43.677016,-79.389367
2,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
3,Tinuno,Filipino Restaurant,43.671281,-79.374920
4,Pie Squared,Pie Shop,43.672143,-79.377856
5,Manulife Financial,Office,43.672140,-79.382255
6,Starbucks,Coffee Shop,43.671478,-79.380664
7,Nijo Japanese Restaurant,Japanese Restaurant,43.671849,-79.378824
8,Craigleigh Gardens,Park,43.678099,-79.371586
9,Booster Juice,Smoothie Shop,43.671566,-79.378581


In [53]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


In [ ]:
Only one venue was returned

Use the function to repeat process to find out the neighborhoods in Downtown Toronto


In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
creating a new dataframe called downtown_venues

In [55]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )


Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, Richmond, King
Toronto Islands, Harbourfront East, Union Station
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Grange Park, Chinatown, Kensington Market
Island airport, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, King and Spadina, CN Tower
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


Analyzing neighborhoods

In [56]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

grouping rows by neighborhoods and by taking the mean of the frequency of occurrence of each category


In [57]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, Richmond, King",0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.00,0.00,0.00,0.04,0.0,0.000000,0.00,0.00,0.0,0.04,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.02,0.04,0.0,0.00,0.02,0.02,0.0,0.06,0.0,0.00,0.00,0.0,0.0,0.00,0.02,0.060000,0.0,0.0,0.0,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.04,0.00,0.00,0.02,0.0,0.0,0.0,0.02,0.0,0.00,0.02,0.0,0.0,0.0,0.0,0.00,0.0,0.04,0.0,0.00,0.0,0.000000,0.02,0.02,0.0,0.00,0.0,0.0,0.0,0.00,0.02,0.0,0.0,0.00,0.0,0.0,0.00,0.02,0.0,0.00,0.0,0.0,0.000000,0.02,0.0,0.02,0.0,0.000000,0.0,0.0,0.0,0.00,0.04,0.0,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.02,0.020000,0.00,0.02,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.02,0.0,0.00,0.06,0.0,0.02,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.00,0.00
1,Berczy Park,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.02,0.00,0.00,0.00,0.0,0.000000,0.02,0.04,0.0,0.00,0.02,0.02,0.02,0.0,0.02,0.02,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.04,0.0,0.04,0.00,0.0,0.0,0.02,0.06,0.080000,0.0,0.0,0.0,0.02,0.00,0.02,0.000000,0.02,0.02,0.00,0.00,0.00,0.00,0.020000,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.04,0.02,0.0,0.00,0.0,0.02,0.02,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.00,0.0,0.020000,0.00,0.02,0.0,0.00,0.0,0.0,0.0,0.02,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.02,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.020000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.04,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0

Analysis each neighborhood along with the top 3 most common venues

In [58]:
num_top_venues = 3

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
                 venue  freq
0  American Restaurant  0.06
1                 Café  0.06
2          Coffee Shop  0.06


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.06
2  Cocktail Bar  0.06


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.14
1  Bubble Tea Shop  0.06
2     Burger Joint  0.06


----Christie----
           venue  freq
0  Grocery Store  0.20
1           Café  0.20
2           Park  0.13


----Church and Wellesley----
          venue  freq
0  Burger Joint  0.06
1   Coffee Shop  0.06
2   Men's Store  0.04


----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop  0.14
1        Hotel  0.10
2         Café  0.10


----First Canadian Place, Underground city----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.10
2        Hotel  0.06


----Garden District, Ryerson----
            venue  freq
0  Clothing Store  0.08
1            Café  0.08
2     Coffee Shop

sorting the venues in descending order

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the dataframe and show up the top 5 venues for each neighborhood.


In [60]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, Richmond, King",Café,American Restaurant,Steakhouse,Coffee Shop,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café
2,Central Bay Street,Coffee Shop,Café,Burger Joint,Bubble Tea Shop,Italian Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Restaurant
4,Church and Wellesley,Coffee Shop,Burger Joint,Japanese Restaurant,Restaurant,Men's Store
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Gastropub,Deli / Bodega
6,"First Canadian Place, Underground city",Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega
7,"Garden District, Ryerson",Clothing Store,Café,Coffee Shop,Middle Eastern Restaurant,Theater
8,"Grange Park, Chinatown, Kensington Market",Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery
9,"Harbourfront, Regent Park",Coffee Shop,Café,Bakery,Park,Pub


In [ ]:
Using k-means to cluster the neighborhood into 5 clusters

In [62]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 4, 3, 1, 1, 3, 3, 3], dtype=int32)

creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [63]:
downtown_merged = downtown_data

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Cosmetics Shop,Dog Run
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,3,Coffee Shop,Restaurant,Pub,Indian Restaurant,Italian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Coffee Shop,Burger Joint,Japanese Restaurant,Restaurant,Men's Store
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,4,Coffee Shop,Café,Bakery,Park,Pub
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Café,Coffee Shop,Middle Eastern Restaurant,Theater


Visualize results

In [64]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [65]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Deli / Bodega,Gastropub


Cluster 2

In [66]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Cosmetics Shop,Dog Run
5,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Gastropub,Gym,Hotel
6,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café
15,Downtown Toronto,1,Italian Restaurant,Cocktail Bar,Restaurant,Café,Pub


Cluster 3

In [67]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Downtown Toronto,2,Coffee Shop,Café,Hotel,Gastropub,Deli / Bodega


Cluster 4

In [68]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Downtown Toronto,3,Coffee Shop,Restaurant,Pub,Indian Restaurant,Italian Restaurant
2,Downtown Toronto,3,Coffee Shop,Burger Joint,Japanese Restaurant,Restaurant,Men's Store
4,Downtown Toronto,3,Clothing Store,Café,Coffee Shop,Middle Eastern Restaurant,Theater
7,Downtown Toronto,3,Coffee Shop,Café,Burger Joint,Bubble Tea Shop,Italian Restaurant
8,Downtown Toronto,3,Café,American Restaurant,Steakhouse,Coffee Shop,Hotel
9,Downtown Toronto,3,Coffee Shop,Aquarium,Scenic Lookout,Park,Café
12,Downtown Toronto,3,Café,Coffee Shop,Bakery,Bar,Bookstore
13,Downtown Toronto,3,Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery
14,Downtown Toronto,3,Airport Terminal,Airport Lounge,Airport Service,Boat or Ferry,Plane
16,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega


Cluster 5

In [69]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Downtown Toronto,4,Coffee Shop,Café,Bakery,Park,Pub
